# Hybrid RAG with Azure Data Explorer  

You can run this notebook after succesfully running these notebooks: 
- "1- Create entities and embeddings" 
- "2- Save entities and embeddings into ADX"

In [4]:
from dotenv import load_dotenv
import os
import textwrap
import pandas as pd
from IPython.display import display, HTML, JSON, Markdown, Image

from openai import AzureOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from tenacity import retry, wait_random_exponential, stop_after_attempt

from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table


# Configure environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_GPT4_32k_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT4_32k_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
api_version = "2024-02-01"

AAD_TENANT_ID = os.getenv("AAD_TENANT_ID")
KUSTO_CLUSTER = os.getenv("KUSTO_CLUSTER")
KUSTO_DATABASE = os.getenv("KUSTO_DATABASE")
KUSTO_TABLE = os.getenv("KUSTO_TABLE")
KUSTO_MANAGED_IDENTITY_APP_ID = os.getenv("KUSTO_MANAGED_IDENTITY_APP_ID")
KUSTO_MANAGED_IDENTITY_SECRET = os.getenv("KUSTO_MANAGED_IDENTITY_SECRET")

# define embeddings 
embeddings = AzureOpenAIEmbeddings(
    model=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME,
    azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
    openai_api_version=api_version,
    chunk_size = 1
)

# call OpenAI to get the answer
clientOpenAI = AzureOpenAI(
  azure_endpoint = OPENAI_DEPLOYMENT_ENDPOINT, 
  api_key=OPENAI_API_KEY,  
  api_version="2023-05-15"
)


In [5]:
#we use the tenacity library to create delays and retries when calling openAI embeddings to avoid hitting throttling limits
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def call_openAI(messages):
    response = clientOpenAI.chat.completions.create(
        model=OPENAI_GPT4_32k_DEPLOYMENT_NAME,
        messages = messages,
        temperature=0
    )

    return response.choices[0].message.content

#we use the tenacity library to create delays and retries when calling openAI embeddings to avoid hitting throttling limits
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def calc_embeddings(text):
    deployment = OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME
    # replace newlines, which can negatively affect performance.
    txt = text.replace("\n", " ")
    return embeddings.embed_query(txt)



def call_openAI_for_final_answer(question, answer):
    prompt = 'Question: {}'.format(question) + '\n' + 'Information: {}'.format(answer)
    # prepare prompt
    messages = [{"role": "system", "content": "You are a HELPFUL assistant answering users questions. Answer the question using the provided information and do not add anything else."},
            {"role": "user", "content": prompt}]
    return call_openAI(messages)

def prettyprint(text: str) -> str:
    print(textwrap.fill(text, 60))

In [60]:
# Connect to adx using AAD app registration
cluster = KUSTO_CLUSTER
kcsb = KustoConnectionStringBuilder.with_aad_application_key_authentication(cluster, KUSTO_MANAGED_IDENTITY_APP_ID, KUSTO_MANAGED_IDENTITY_SECRET,  AAD_TENANT_ID)
client = KustoClient(kcsb)
kusto_db = KUSTO_DATABASE
embeddings_table = "aviationIncidentsEmbeddings"
entities_table = "aviationIncidentsEntities"

In [47]:
def get_embeddings_from_adx(question, nr_of_answers=1):
        searchedEmbedding = calc_embeddings(question)
        kusto_query = embeddings_table + " | extend similarity = series_cosine_similarity_fl(dynamic("+str(searchedEmbedding)+"), embedding,1,1) | top " + str(nr_of_answers) + " by similarity desc "
        response = client.execute(kusto_db, kusto_query)

        for row in response.primary_results[0]:
                return row['content']

In [48]:
def answer_in_NL_withOpenAI(question, nr_of_answers=2):
    answer_from_ADX = get_embeddings_from_adx(question,nr_of_answers)
    prompt = 'Question: {}'.format(question) + '\n' + 'Information: {}'.format(answer_from_ADX)
    # prepare prompt
    messages = [{"role": "system", "content": """You are a HELPFUL assistant answering users questions. 
                Answer the question using the provided information only and do not add anything else.
                """},
                {"role": "user", "content": prompt}]

    result = call_openAI(messages)
    display(HTML(result))

In [69]:
#asking Global questions using embeddings - we get only 2 answers - we cannot ask Global questions since we are limited
# to 2 answers
answer_in_NL_withOpenAI("List all the accident numbers for the aviation reports occurred in the USA during Phase of Operation: landing?", 2)

In [41]:
#let's use GPT4 to translate the question to KQL
def generate_KQL_query(question):
    question_formatted = f"""You are a HELPFUL assistant translating Natural language to KQL queries. 
               
                The Table in the database is called "aviationIncidentsEntities" and it has the following fields:
                'document_id','document_name', 'entities'
                The 'entities' field is a KQL dynamic field that contains the following properties:
                aircraft_make:'',accident_number:'',aircraft_damage:'',city:'', state:'', country:'', phase_of_operation:'', pilot_flight_hours:'', engine_manufacturer:''
                
                Answer with the KQL query ONLY and do not add anything else.
                """
    messages = [{"role": "system", "content": question_formatted},
                {"role": "user", "content": question}]

    result = call_openAI(messages)
    return result

In [71]:
#Now let's combine ADX search and embeddings to answer the question
query = generate_KQL_query("List all the accidents numbers occurred in the USA?")
print("KQL query: " + query)

#run the KQL query
response = client.execute(kusto_db, query)
nr_of_results = len(response.primary_results[0])
print("Number of results: " + str(nr_of_results))

#extract the accident numbers
list_of_accident_numbers = ''
for row in response.primary_results[0]:
    txt = (row["entities_accident_number"])
    list_of_accident_numbers += txt + ', '
print("List of accident numbers: " + list_of_accident_numbers)

KQL query: aviationIncidentsEntities
| where entities.country == "USA"
| project entities.accident_number
Number of results: 6
List of accident numbers: NYC08CA055, SEA08CA049, SEA08CA052, NYC08CA040, NYC08CA073, NYC08CA068, 


In [74]:
answer_in_NL_withOpenAI("List all the accident numbers for the aviation reports occurred during Phase of Operation: landing from the following list: " + list_of_accident_numbers, nr_of_results)